In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn import metrics
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
caminho = r"/content/drive/MyDrive/8ºSEMESTRE/TG/LightGBM/base.parquet"
base = pd.read_parquet(caminho)

In [ ]:
base['var_325'] = base['var_325'].astype('category')
base['var_1'] = base['var_1'].astype('category')
base['var_339'] = base['var_339'].astype('category')
base['CAT_var_275'] = base['CAT_var_275'].astype('category')
base['var_365'] = base['var_365'].astype('category')
# base['var_resposta'] = base['var_resposta'].astype('category')

In [ ]:
base["var_resposta"] = 1-base["var_resposta"]

base["var_resposta"]= base["var_resposta"].astype('category')

In [ ]:
base["var_resposta"]

,var_resposta
0,1
1,1
2,1
3,1
4,1
...,...
411466,1
411467,1
411468,0
411469,1


In [ ]:
treino = base[base['split'] == "Treinamento"]
import random
np.random.seed(290)
split = np.random.choice(['Treinamento', 'Validacao'], size = len(treino), p = [0.8, 0.2])

# Adicionando a coluna 'split' na base de dados
treino['split'] = split
print(treino['split'].value_counts())

split
Treinamento    263227
Validacao       65774
Name: count, dtype: int64


<ipython-input-37-edcd998e3244>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treino['split'] = split


In [ ]:
treinof = treino[treino['split'] == "Treinamento"]
treinof = treinof.drop(columns = ["id","amostra","foto","split"])

X_treino = treinof.drop(["var_resposta"],axis=1)
Y_treino = treinof["var_resposta"]
print(X_treino.shape)

(263227, 213)


In [ ]:
valid = treino[treino['split'] == "Validacao"]
valid = valid.drop(columns = ["id","amostra","foto","split"])


X_valid = valid.drop(["var_resposta"],axis=1)
Y_valid = valid["var_resposta"]
print(X_valid.shape)

(65774, 213)


In [ ]:
teste = base[base['split'] == "Teste"]
teste = teste.drop(columns = ["id","amostra","foto","split"])
X_teste = teste.drop(["var_resposta"],axis=1)
Y_teste = teste["var_resposta"]
print(X_teste.shape)

(82470, 213)


Otimização Baeysiana

In [ ]:
# !pip install bayes_opt
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
from xgboost import XGBClassifier
import optuna
from sklearn import metrics

def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': 1.0,  # Ajustado para datasets grandes
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50)
    }
    class_xgb = XGBClassifier(**params,enable_categorical=True)
    class_xgb.fit(X_treino, Y_treino)
    predicoes = class_xgb.predict_proba(X_valid)[:, 1]
    auc = metrics.roc_auc_score(Y_valid, predicoes)
    return auc

In [ ]:
# Criando o estudo e otimizando
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2024-09-10 13:34:04,675] A new study created in memory with name: no-name-08fd53ba-21d8-4427-a329-085970eef82e
[I 2024-09-10 13:36:17,970] Trial 0 finished with value: 0.815837973756939 and parameters: {'learning_rate': 0.10242950441585301, 'n_estimators': 834, 'max_depth': 4, 'colsample_bytree': 0.5379808560329132, 'gamma': 1.6498384136679611, 'min_child_weight': 39}. Best is trial 0 with value: 0.815837973756939.
[I 2024-09-10 13:37:17,896] Trial 1 finished with value: 0.7697420418737699 and parameters: {'learning_rate': 0.6218244140692816, 'n_estimators': 356, 'max_depth': 7, 'colsample_bytree': 0.17027093851018366, 'gamma': 2.5725996533154865, 'min_child_weight': 8}. Best is trial 0 with value: 0.815837973756939.
[I 2024-09-10 13:40:59,972] Trial 2 finished with value: 0.8181209065192219 and parameters: {'learning_rate': 0.01807591176318983, 'n_estimators': 610, 'max_depth': 8, 'colsample_bytree': 0.41866547374554636, 'gamma': 1.5273505997688512, 'min_child_weight': 32}. Best is

In [ ]:
print('Melhores hiperparâmetros:', study.best_params)
print('Melhor AUC:', study.best_value)
# Melhores hiperparâmetros: {'learning_rate': 0.015605121823552684, 'n_estimators': 827, 'max_depth': 10, 'colsample_bytree': 0.24819761853105432, 'gamma': 3.4546190502741627, 'min_child_weight': 34}
# Melhor AUC: 0.8183600890437032

Melhores hiperparâmetros: {'learning_rate': 0.015605121823552684, 'n_estimators': 827, 'max_depth': 10, 'colsample_bytree': 0.24819761853105432, 'gamma': 3.4546190502741627, 'min_child_weight': 34}
Melhor AUC: 0.8183600890437032


In [ ]:
!pip install bayesian-optimization
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.9 MB/s eta 0:00:00


In [ ]:
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn import metrics

# Função para ajuste do modelo com parâmetros específicos
def modelFitter(learning_rate, n_estimators, max_depth, colsample_bytree, min_child_weight, gamma):
    # Defina os parâmetros do modelo XGBoost com os valores fornecidos pelo otimizador
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "learning_rate": learning_rate,
        "n_estimators": int(n_estimators),
        "max_depth": int(max_depth),
        "colsample_bytree": colsample_bytree,
        "min_child_weight": int(min_child_weight),
        "gamma": gamma,
        "verbose": -1
    }

    # Inicializa o classificador XGBoost com os parâmetros
    class_xgb = xgb.XGBClassifier(**params,enable_categorical=True)
    class_xgb.fit(X_treino, Y_treino)

    # Faz previsões no conjunto de validação
    predicoes = class_xgb.predict_proba(X_valid)[:, 1]

    # Calcula o AUC
    auc = metrics.roc_auc_score(Y_valid, predicoes)
    return auc

# Define os limites dos hiperparâmetros para a otimização bayesiana
params = {
    "learning_rate": (0.01, 1.0),
    "n_estimators": (100, 1000),
    "max_depth": (3, 10),
    "colsample_bytree": (0.05, 1.0),
    "min_child_weight": (1, 50),
    "gamma": (0, 5)
}

# Inicializa a otimização bayesiana
optimizer = BayesianOptimization(
    f=modelFitter,
    pbounds=params,
    random_state=1
)

# Executa a otimização
optimizer.maximize(init_points=15, n_iter=15)


|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:38:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


| 1         | 0.8003    | 0.4462    | 3.602     | 0.01011   | 5.116     | 8.191     | 183.1     |


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:39:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


| 2         | 0.7958    | 0.2269    | 1.728     | 0.4028    | 6.772     | 21.54     | 716.7     |


...

| 28        | 0.7817    | 0.4945    | 3.897     | 0.7585    | 6.953     | 45.07     | 364.8     |


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:23:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


| 29        | 0.8177    | 0.8914    | 4.803     | 0.01258   | 8.306     | 48.72     | 772.8     |


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:27:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


| 30        | 0.7946    | 0.2641    | 4.187     | 0.5809    | 7.696     | 48.87     | 772.8     |


In [ ]:
# Best parameters and AUC score
print(optimizer.max)

{'target': 0.8185400966096508, 'params': {'colsample_bytree': 0.3233865716570313, 'gamma': 0.6501428605913884, 'learning_rate': 0.0291732882915941, 'max_depth': 7.751848730579237, 'min_child_weight': 11.369777684002893, 'n_estimators': 338.99199343500356}}


In [ ]:
optimizer.max['params']

{'colsample_bytree': 0.3233865716570313,
 'gamma': 0.6501428605913884,
 'learning_rate': 0.0291732882915941,
 'max_depth': 7.751848730579237,
 'min_child_weight': 11.369777684002893,
 'n_estimators': 338.99199343500356}

In [ ]:
Algortimo Genético

Modelagem

In [ ]:
# from xgboost import XGBClassifier
# class_xgb = XGBClassifier(learning_rate= 0.06426915014337858,
#                                 colsample_bytree= 0.6421921100022127,
#                                 max_depth= 14,
#                                 n_estimators= 76,
#                                enable_categorical=True)
# class_xgb.fit(X_treino,Y_treino)
from xgboost import XGBClassifier

#study.best_params
class_xgb = XGBClassifier(colsample_bytree= 0.3233865716570313,
                           gamma= 0.6501428605913884,
                           learning_rate= 0.0291732882915941,
                           max_depth= 78,
                           min_child_weight= 11.369777684002893,
                           n_estimators= 339,
                          enable_categorical=True)
class_xgb.fit(X_treino,Y_treino)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3233865716570313, device=None,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric=None, feature_types=None, gamma=0.6501428605913884,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0291732882915941,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=78, max_leaves=None,
              min_child_weight=11.369777684002893, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=339,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [ ]:
predicoes = class_xgb.predict_proba(X_teste)[:, 1]
predicoes


array([0.99467176, 0.98638654, 0.9277757 , ..., 0.99470985, 0.9988888 ,
       0.9973189 ], dtype=float32)

In [ ]:
auc = metrics.roc_auc_score(Y_teste, predicoes)
auc #0.816247003340626

0.8108760701121677